In [19]:
import random

import numpy as np
import yaml
import torch
from experiment import EXPERIMENT_CATALOG
from utils import Task, get_text

In [2]:
with open('configs/pretrain_baseline.yaml') as fin:
    config = yaml.safe_load(fin)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
experiment = EXPERIMENT_CATALOG['pretrain_baseline'](config, device)


/home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legac

In [3]:
model = experiment.model
test_iter = experiment.task.test
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5851, 512)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(512, 256, num_layers=2, dropout=0.5)
    (out): Linear(in_features=256, out_features=5851, bias=True)
  )
  (decoder): AttnDecoder(
    (embedding): Embedding(4244, 512)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(512, 256, num_layers=2, dropout=0.5)
    (attention): Attention(
      (attn_combine): Linear(in_features=512, out_features=256, bias=True)
      (tanh): Tanh()
    )
    (out): Linear(in_features=256, out_features=4244, bias=True)
  )
)

In [4]:
src_str = experiment.source.tokenize('В стильном ресторане отеля подают блюда средиземноморской кухни из органических местных продуктов.')
src = experiment.source.process([src_str]).to(device)
trg_str = experiment.target.tokenize('Organic, locally sourced ingredients are blended in the Mediterranean dishes of its stylish restaurant.')
trg = experiment.target.process([trg_str]).to(device)

In [16]:
batch_size = 1
max_len = trg.shape[0]
trg_vocab_size = model.decoder.output_dim

# tensor to store decoder outputs
outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(device)

# last hidden state of the encoder is used as the initial hidden state of the decoder
enc_output = model.encoder(src)
enc_outputs, hidden, cell = enc_output['rnn_out'], enc_output['rnn_hidden'], enc_output['rnn_cell']

# first input to the decoder is the <sos> tokens
dec_input = trg[0, :]
attn_list = []
for t in range(1, max_len):
    dec_output = model.decoder(dec_input, hidden, cell, enc_outputs)
    output, hidden, cell = dec_output['prediction'], dec_output['rnn_hidden'], dec_output['rnn_cell']
    attn_list.append(dec_output['attention_map'].cpu().detach().numpy()[0])
    outputs[t] = output
    teacher_force = random.random() < 0.
    top1 = output.max(1)[1]
    dec_input = (top1 if teacher_force else trg[t])

In [17]:
attn_list = np.array(attn_list)

In [18]:
src_os = ['<sos>', *src_str, '<eos>']
trg_os = ['<sos>', *trg_str, '<eos>']
print('target', 'top1', 'top2', 'top3', sep='\t\t')
for i, attn_mask in enumerate(attn_list[:, :, 1]):
    top_n = attn_mask.argsort()[-3:]
    print(trg_os[i], *[src_os[j] for j in top_n], sep='\t\t')
# [(trg_os[i], src_os[attn]) for i, attn in enumerate(np.argmax(a, axis=1)[:, 1])]

target		top1		top2		top3
<sos>		кухни		подают		блюда
organic		органических		из		кухни
,		органических		кухни		местных
locally		продуктов		органических		местных
sourced		кухни		органических		местных
ingredients		продуктов		местных		органических
are		.		продуктов		кухни
blended		.		продуктов		отеля
in		подают		ресторане		отеля
the		ресторане		подают		отеля
mediterranean		блюда		подают		отеля
dishes		блюда		подают		отеля
of		кухни		подают		отеля
its		блюда		подают		отеля
stylish		отеля		подают		блюда
restaurant		<eos>		подают		отеля
.		.		кухни		<eos>


In [35]:
original_text = []
generated_text = []
source_text = []
source_vocab = experiment.source.vocab 
target_vocab = experiment.target.vocab
with torch.no_grad():
    for i, batch in enumerate(test_iter):
        src = batch.src
        trg = batch.trg

        output = model(src, trg, 1.)  # turn on teacher forcing

        # trg = [trg sent len, batch size]
        # output = [trg sent len, batch size, output dim]
        output = output.argmax(dim=-1)

        original_text.extend([get_text(x, target_vocab) for x in trg.cpu().numpy().T])
        generated_text.extend([get_text(x, target_vocab) for x in output[1:].detach().cpu().numpy().T])
        source_text.extend([get_text(x, source_vocab) for x in src.cpu().numpy().T])

/home/cherepaha/miniconda/envs/py37/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [37]:
sample_i = random.sample(range(len(original_text)), 3)
for i in sample_i:
    print('Source:', ' '.join(source_text[i]))
    print('Original:', ' '.join(original_text[i]))
    print('Generated:', ' '.join(generated_text[i]))

Source: гости смогут приготовить барбекю на открытом воздухе и приятно посидеть в беседке , а также сыграть в настольный теннис в комнате для игр с стойкой .
Original: enjoy a meal using the bbq facilities , located outdoors in the covered . you can play table tennis in the games room , which also features a built - in bar .
Generated: guests can enjoy a the outdoor pool and table tennis , and table tennis at the lobby and , and the lobby room .
Source: во всех апартаментах и номерах есть телевизор со спутниковыми каналами , гостиный уголок и ванная комната с душем или ванной .
Original: the accommodation units feature a satellite tv . each unit has a bathroom with a shower or a bath .
Generated: all units feature a satellite tv , a seating area and a private bathroom with a bath or shower .
Source: отель расположен на - стрит в и располагает ресторанами , баром и фитнес - центром .
Original: on dublin ’ s street , the has fantastic restaurants , a bar and a fitness suite .
Generated: 